In [15]:
import os
import pandas as pd
from datetime import datetime, timedelta


In [2]:
project_dir = '/data/konrad/workspace'

dir_path = project_dir + "/datasets/calfs_observations/feeder"
csv_files = [d for d in os.listdir(dir_path) if d.endswith("csv") and os.path.isfile(os.path.join(dir_path, d))]

def open_feeder(csv_path, columns):
    aptms = pd.read_csv(csv_path, delimiter=';', header=0, names=columns).reset_index(drop=True)
    aptms["start_time"] = pd.to_timedelta(aptms["start visit"])
    aptms["end_time"] = pd.to_timedelta(aptms["end visit"])
    aptms["timedelta"] = (aptms["end_time"] - aptms["start_time"]).apply(lambda x: x.total_seconds())

    aptms = aptms[aptms["timedelta"] > 0]
    aptms = aptms[aptms["start_time"] < aptms["end_time"]]
    return aptms

In [3]:
columns = ['date', 'start visit', 'end visit', 'animal index', 'animal no.',
       'group', 'station', 'housing weight', 'entitlement act.', 'feed mode',
       'next entitlement at', 'cons. today ml', 'cons. today %',
       'add. 1 today', 'add. 2 today', 'feed speed ml/min', 'visit w. entit.',
       'visit wo. entit.', 'break wo. add.', 'break w. add.', 'total MP g.',
       'total milk', 'failure', 'warnings']
impt_days = ["2022_08_01", "2022_08_02", "2022_08_03", "2022_08_19", "2022_08_20", "2022_08_21", "2022_08_22", "2022_08_23", "2022_08_24", "2022_08_26", "2022_09_19" 
            , "2022_09_20", "2022_09_21", "2022_09_22", "2022_09_23", "2022_09_24", "2022_09_25", "2022_09_26", "2022_09_27", "2022_09_28"]
minutes = 10
stations = [1, 2]

In [4]:
records = []
whole_apptms = []
for i, d in enumerate(csv_files):

    if not any([day in d for day in impt_days]):
        continue
        
    csv_path = os.path.join(dir_path, d)
    aptms = pd.read_csv(csv_path, delimiter=';', header=0, names=columns).reset_index(drop=True)

    whole_apptms.append(aptms)

whole_apptms = pd.concat(whole_apptms, axis=0)
whole_apptms.to_csv('/data/data_calves/konrad/whole_feeder_aptms.csv', index=False)

In [7]:
whole_apptms[['date', 'start visit', 'end visit', 'animal index', 'animal no.',
       'group', 'station',]].head()

,date,start visit,end visit,animal index,animal no.,group,station
0,2022/08/26,00:04:13,00:04:19,127,4159,A,2
1,2022/08/26,00:04:20,00:04:58,127,4159,A,2
2,2022/08/26,00:10:35,00:10:59,81,4156,A,2
3,2022/08/26,00:12:27,00:12:59,81,4156,A,2
4,2022/08/26,00:12:59,00:13:11,81,4156,A,2


In [30]:
# Check for consecutive rows meeting the criteria
df = whole_apptms.reset_index(drop=True)

# Function to convert time strings to datetime objects
def to_datetime(date_str, time_str):
    return datetime.strptime(f"{date_str} {time_str}", "%Y/%m/%d %H:%M:%S")

for i in range(len(df) - 1):
    start_time1 = to_datetime(df.loc[i, 'date'], df.loc[i, 'start visit'])
    start_time2 = to_datetime(df.loc[i + 1, 'date'], df.loc[i + 1, 'start visit'])
    
    # if (df.loc[i, 'animal no.'] == 4151 and df.loc[i + 1, 'animal no.'] == 4147 and 
    #     df.loc[i, 'station'] == 2 and df.loc[i + 1, 'station'] == 2 and 
    #     df.loc[i, 'date'] == df.loc[i + 1, 'date']):
        
    # if (df.loc[i, 'animal no.'] == 4151 and df.loc[i + 1, 'animal no.'] == 4147):
    if (df.loc[i, 'animal no.'] == 4147 and df.loc[i + 1, 'animal no.'] == 4152):
        
        print(f"try {(start_time2 - start_time1)}, {start_time2}, {start_time1} between {i} and {i+1}")
        
        if (start_time2 - start_time1) >= timedelta(minutes=2):
            print(f"Found at index {i} and {i + 1}")
            break
else:
    print("No matching consecutive rows found.")

try 0:00:30, 2022-08-02 05:35:06, 2022-08-02 05:34:36 between 1119 and 1120
try 0:00:19, 2022-08-02 11:44:27, 2022-08-02 11:44:08 between 1271 and 1272
try 0:00:25, 2022-08-01 11:05:59, 2022-08-01 11:05:34 between 8919 and 8920
try -1 day, 23:55:15, 2022-08-01 11:31:13, 2022-08-01 11:35:58 between 8926 and 8927
try 0:04:10, 2022-08-01 18:19:37, 2022-08-01 18:15:27 between 9051 and 9052
Found at index 9051 and 9052


In [29]:
df.iloc[8916:8936, :]

,date,start visit,end visit,animal index,animal no.,group,station,housing weight,entitlement act.,feed mode,...,add. 2 today,feed speed ml/min,visit w. entit.,visit wo. entit.,break wo. add.,break w. add.,total MP g.,total milk,failure,warnings
8916,2022/08/01,10:46:51,10:55:31,133,4164,A,1,0,2000,from,...,0,70,2,0,1,0,1071,0,0,0
8917,2022/08/01,10:58:41,10:58:50,109,4146,A,2,0,0,till,...,0,0,0,15,0,0,11908,0,0,0
8918,2022/08/01,10:59:56,11:02:21,115,4157,A,1,0,2075,from,...,0,480,3,0,2,0,49783,0,0,0
8919,2022/08/01,11:05:34,11:06:05,111,4147,A,2,0,0,till,...,0,0,0,23,0,0,13668,0,0,0
8920,2022/08/01,11:05:59,11:09:30,78,4152,A,1,0,2500,max.,...,0,574,2,2,0,0,31858,0,0,0
8921,2022/08/01,11:19:30,11:24:39,78,4152,A,1,0,2000,max.,...,0,574,3,2,1,0,31858,0,0,0
8922,2022/08/01,11:24:40,11:26:50,131,4162,A,1,0,2000,from,...,0,250,6,0,3,0,7845,0,0,0
8923,2022/08/01,11:26:50,11:29:47,78,4152,A,1,0,2000,max.,...,0,574,4,2,2,0,31858,0,0,0
8924,2022/08/01,11:29:48,11:31:13,127,4159,A,1,0,1575,from,...,0,309,8,0,7,0,20666,0,0,0
8925,2022/08/01,11:35:17,11:35:27,109,4146,A,2,0,0,till,...,0,0,0,16,0,0,11908,0,0,0


In [28]:
df.iloc[9040:9060, :]

,date,start visit,end visit,animal index,animal no.,group,station,housing weight,entitlement act.,feed mode,...,add. 2 today,feed speed ml/min,visit w. entit.,visit wo. entit.,break wo. add.,break w. add.,total MP g.,total milk,failure,warnings
9040,2022/08/01,17:42:59,17:43:09,131,4162,A,1,0,0,BLOCKED,...,0,218,9,0,5,0,8426,0,0,0
9041,2022/08/01,17:43:12,17:44:35,131,4162,A,1,0,0,BLOCKED,...,0,218,9,0,5,0,8426,0,0,0
9042,2022/08/01,17:44:35,17:45:14,131,4162,A,1,0,0,BLOCKED,...,0,218,9,0,5,0,8426,0,0,0
9043,2022/08/01,17:47:27,17:52:35,77,4151,A,2,0,2500,BLOCKED,...,0,654,8,0,1,0,63359,0,0,0
9044,2022/08/01,17:52:35,17:52:51,77,4151,A,2,0,0,BLOCKED,...,0,654,8,0,1,0,63359,0,0,0
9045,2022/08/01,17:55:51,18:01:41,127,4159,A,1,0,2000,from,...,0,328,10,0,9,0,21026,0,0,0
9046,2022/08/01,18:08:14,18:13:13,132,4163,A,1,0,2000,BLOCKED,...,0,474,8,3,5,0,3776,0,0,0
9047,2022/08/01,18:13:14,18:13:45,132,4163,A,1,0,0,BLOCKED,...,0,474,8,3,5,0,3776,0,0,0
9048,2022/08/01,18:13:45,18:13:53,132,4163,A,1,0,0,BLOCKED,...,0,474,8,3,5,0,3776,0,0,0
9049,2022/08/01,18:13:53,18:13:59,132,4163,A,1,0,0,BLOCKED,...,0,474,8,3,5,0,3776,0,0,0


In [26]:
df.iloc[9000:9020, :]

,date,start visit,end visit,animal index,animal no.,group,station,housing weight,entitlement act.,feed mode,...,add. 2 today,feed speed ml/min,visit w. entit.,visit wo. entit.,break wo. add.,break w. add.,total MP g.,total milk,failure,warnings
9000,2022/08/01,16:23:26,16:25:51,81,4156,A,2,0,3000,max.,...,0,435,10,0,5,0,45837,0,0,0
9001,2022/08/01,16:26:25,16:27:23,111,4147,A,2,0,0,till,...,0,0,0,41,0,0,13668,0,0,0
9002,2022/08/01,16:27:37,16:27:55,81,4156,A,2,0,2450,max.,...,0,435,11,0,5,0,45837,0,0,0
9003,2022/08/01,16:27:59,16:28:05,81,4156,A,2,0,2450,max.,...,0,435,12,0,5,0,45837,0,0,0
9004,2022/08/01,16:28:35,16:29:08,111,4147,A,2,0,0,till,...,0,0,0,41,0,0,13668,0,0,0
9005,2022/08/01,16:29:08,16:29:30,111,4147,A,2,0,0,till,...,0,0,0,41,0,0,13668,0,0,0
9006,2022/08/01,16:25:32,16:30:22,127,4159,A,1,0,2000,from,...,0,323,9,0,8,0,20842,0,0,0
9007,2022/08/01,16:38:25,16:40:33,77,4151,A,2,0,2500,max.,...,0,652,3,0,1,0,62919,0,0,0
9008,2022/08/01,16:53:42,16:54:46,111,4147,A,2,0,0,till,...,0,0,0,42,0,0,13668,0,0,0
9009,2022/08/01,16:54:53,16:55:06,111,4147,A,2,0,0,till,...,0,0,0,42,0,0,13668,0,0,0


In [51]:
records = []
for d in csv_files:

    if not any([day in d for day in impt_days]):
        continue
        
    csv_path = os.path.join(dir_path, d)
    aptms = pd.read_csv(csv_path, delimiter=';', header=0, names=columns).reset_index(drop=True)
    # print(aptms.shape, csv_path)
    # print(aptms.columns)
    aptms["start_time"] = pd.to_timedelta(aptms["start visit"])
    aptms["end_time"] = pd.to_timedelta(aptms["end visit"])
    aptms["timedelta"] = (aptms["end_time"] - aptms["start_time"]).apply(lambda x: x.total_seconds())

    aptms = aptms[aptms["timedelta"] > 0]
    aptms = aptms[aptms["start_time"] < aptms["end_time"]]
    # print(aptms[aptms["timedelta"] <= 0].shape)
    
    # for idx in aptms.sort_values(by="start_time").index.values:
    # print(aptms.index.values)
    for idx, row in aptms.iterrows():
        # key = "{day}_{parc}".format(day=aptms.iloc[idx]["start_time"], parc=aptms.iloc[idx]["station"])
        # row = aptms.iloc[idx]
        # key = "{day}".format(day=row["start_time"])

        after_at =  row["start_time"] + pd.Timedelta(seconds=minutes * 60)
        before_at =  row["start_time"] - pd.Timedelta(seconds=minutes * 60)

        for station in stations:
            records.append({
                "day": d,
                "station": row["station"],
                "in_station": station,
                "start_time": row["start_time"],
                "end_time": row["end_time"],
                "after_at": after_at,
                "before_at": before_at,
                "started_after": aptms[(aptms["station"] == station) & (aptms["start_time"] < after_at) & (aptms["start_time"] > row["start_time"])].shape[0],
                "calf_number_started_after": aptms[(aptms["station"] == station) & (aptms["start_time"] < after_at) & (aptms["start_time"] > row["start_time"])]["animal no."].unique().size,
                "in_after": aptms[(aptms["station"] == station) & (aptms["start_time"] < after_at) & (aptms["end_time"] < after_at) & (aptms["start_time"] > row["start_time"]) & (aptms["end_time"] > row["start_time"])].shape[0],
                "calf_number_in_after": aptms[(aptms["station"] == station) & (aptms["start_time"] < after_at) & (aptms["end_time"] < after_at) & (aptms["start_time"] > row["start_time"]) & (aptms["end_time"] > row["start_time"])]["animal no."].unique().size,
                # "started_before": aptms[(aptms["start_time"] > before_at) & (aptms["start_time"] < row["start_time"])].shape[0],
                # "calf_number_started_before": aptms[(aptms["start_time"] > before_at) & (aptms["start_time"] < row["start_time"])]["animal no."].unique().size,
                # "in_before": aptms[(aptms["start_time"] > before_at) & (aptms["end_time"] > before_at) & (aptms["end_time"] < row["start_time"]) & (aptms["start_time"] < row["start_time"])].shape[0],
                # "calf_number_in_before": aptms[(aptms["start_time"] > before_at) & (aptms["end_time"] > before_at) & (aptms["end_time"] < row["start_time"]) & (aptms["start_time"] < row["start_time"])]["animal no."].unique().size,
            })


    # print((aptms["end_time"] - aptms["start_time"]).head())
    # print((aptms["end_time"].total_seconds()).head())
    # print(aptms[(aptms["end_time"] - aptms["start_time"]).total_seconds() > 0],shape)
records = pd.DataFrame(records)
# records["has_video"] = records.apply(lambda row: any([day in row["day"] for day in impt_days]), axis=1)
filepath = '/data/data_calves/konrad/station_feeder_aptms_metadata.csv'
records.to_csv(filepath, index=False)
records.head()

,day,station,in_station,start_time,end_time,after_at,before_at,started_after,calf_number_started_after,in_after,calf_number_in_after
0,F_2022_08_26.csv,2,1,0 days 00:04:13,0 days 00:04:19,0 days 00:14:13,-1 days +23:54:13,1,1,0,0
1,F_2022_08_26.csv,2,2,0 days 00:04:13,0 days 00:04:19,0 days 00:14:13,-1 days +23:54:13,4,2,4,2
2,F_2022_08_26.csv,2,1,0 days 00:04:20,0 days 00:04:58,0 days 00:14:20,-1 days +23:54:20,1,1,0,0
3,F_2022_08_26.csv,2,2,0 days 00:04:20,0 days 00:04:58,0 days 00:14:20,-1 days +23:54:20,3,1,3,1
4,F_2022_08_26.csv,2,1,0 days 00:10:35,0 days 00:10:59,0 days 00:20:35,0 days 00:00:35,0,0,0,0


In [4]:
filepath = '/data/data_calves/konrad/station_feeder_aptms_metadata.csv'
records = pd.read_csv(filepath)
records["start_time"] = pd.to_timedelta(records["start_time"])
records["end_time"] = pd.to_timedelta(records["end_time"])
records.head()

,day,station,in_station,start_time,end_time,after_at,before_at,started_after,calf_number_started_after,in_after,calf_number_in_after
0,F_2022_08_26.csv,2,1,0 days 00:04:13,0 days 00:04:19,0 days 00:14:13,-1 days +23:54:13,1,1,0,0
1,F_2022_08_26.csv,2,2,0 days 00:04:13,0 days 00:04:19,0 days 00:14:13,-1 days +23:54:13,4,2,4,2
2,F_2022_08_26.csv,2,1,0 days 00:04:20,0 days 00:04:58,0 days 00:14:20,-1 days +23:54:20,1,1,0,0
3,F_2022_08_26.csv,2,2,0 days 00:04:20,0 days 00:04:58,0 days 00:14:20,-1 days +23:54:20,3,1,3,1
4,F_2022_08_26.csv,2,1,0 days 00:10:35,0 days 00:10:59,0 days 00:20:35,0 days 00:00:35,0,0,0,0


In [19]:
# records_sorted = records.sort_values(by=["calf_number_started_after", "started_after"], ascending=False)
# records_sorted[["day", "start_time", "started_after", "in_after", "calf_number_started_after", "calf_number_in_after"]].head(20)

In [5]:
has_records = ['F_2022_08_26.csv', 'F_2022_09_23.csv', 'F_2022_08_01.csv', 'F_2022_08_22.csv', 
       'F_2022_09_22.csv', 
        'F_2022_09_24.csv', 'F_2022_09_21.csv',
       'F_2022_08_21.csv', 'F_2022_08_19.csv', 'F_2022_08_20.csv',
       'F_2022_08_02.csv', 'F_2022_08_03.csv']
records_sorted = records[records["day"].isin(has_records)].copy(deep=True)
records_sorted["start_hour"] = records_sorted["start_time"].apply(lambda x: x.components.hours)
records_sorted = records_sorted[((records_sorted["start_hour"] >= 6) & (records_sorted["start_hour"] < 8)) | (records_sorted["start_hour"] > 18)]
spaced_aptm_with_state = records_sorted.copy(deep=True).reset_index(drop=True)
spaced_aptm_with_state = spaced_aptm_with_state.sort_values(by=["calf_number_started_after"], ascending=False).reset_index(drop=True)
spaced_aptm_with_state[["day", "start_time", "station", "in_station", "started_after", "in_after", "calf_number_started_after", "calf_number_in_after"]].head(20)

,day,start_time,station,in_station,started_after,in_after,calf_number_started_after,calf_number_in_after
0,F_2022_08_03.csv,0 days 19:12:27,1,2,10,9,7,6
1,F_2022_08_21.csv,0 days 19:38:47,2,2,12,11,6,5
2,F_2022_08_03.csv,0 days 19:19:01,2,2,11,10,6,5
3,F_2022_08_19.csv,0 days 19:08:23,2,2,12,11,6,6
4,F_2022_08_19.csv,0 days 19:08:01,2,2,12,11,6,5
5,F_2022_08_21.csv,0 days 19:35:20,2,2,16,15,6,5
6,F_2022_08_21.csv,0 days 19:37:57,2,2,13,12,6,5
7,F_2022_08_21.csv,0 days 19:37:49,1,2,14,13,6,5
8,F_2022_08_21.csv,0 days 19:36:14,2,2,15,14,6,5
9,F_2022_08_03.csv,0 days 19:19:29,2,2,10,9,6,5


In [6]:
has_records = ['F_2022_08_26.csv', 'F_2022_09_23.csv', 'F_2022_08_01.csv', 'F_2022_08_22.csv', 
       'F_2022_09_22.csv', 
        'F_2022_09_24.csv', 'F_2022_09_21.csv',
       'F_2022_08_21.csv', 'F_2022_08_19.csv', 'F_2022_08_20.csv',
       'F_2022_08_02.csv', 'F_2022_08_03.csv']
records_sorted = records[records["day"].isin(has_records)].copy(deep=True)
records_sorted["start_hour"] = records_sorted["start_time"].apply(lambda x: x.components.hours)
records_sorted = records_sorted[((records_sorted["start_hour"] >= 6) & (records_sorted["start_hour"] < 8)) | (records_sorted["start_hour"] > 18)]
spaced_aptm_with_state = records_sorted.copy(deep=True).reset_index(drop=True)


drop_idx=[]
visited_day = {}
minutes = 5
spaced_aptm_with_state["start_time"] = pd.to_timedelta(spaced_aptm_with_state["start_time"])
for idx in spaced_aptm_with_state.sort_values(by="start_time").index.values:
    key = "{day}_{parc}".format(day=spaced_aptm_with_state.iloc[idx]["day"], parc=spaced_aptm_with_state.iloc[idx]["station"])

    if not key in visited_day.keys():
        visited_day[key] = spaced_aptm_with_state.iloc[idx]["start_time"]
        continue
    
    diff = (visited_day[key] - spaced_aptm_with_state.iloc[idx]["start_time"]).total_seconds()
    
    if diff < (minutes * 60):
        drop_idx.append(idx)
    else:
        visited_day[key] = spaced_aptm_with_state.iloc[idx]["start_time"]

spaced_aptm_with_state = spaced_aptm_with_state[~spaced_aptm_with_state.index.isin(drop_idx)].reset_index(drop=True)
spaced_aptm_with_state = spaced_aptm_with_state.sort_values(by=["calf_number_started_after"], ascending=False).reset_index(drop=True)
spaced_aptm_with_state[["day", "start_time", "station", "in_station", "started_after", "in_after", "calf_number_started_after", "calf_number_in_after"]].head(20)

,day,start_time,station,in_station,started_after,in_after,calf_number_started_after,calf_number_in_after
0,F_2022_08_19.csv,0 days 06:04:52,2,2,16,16,3,3
1,F_2022_08_19.csv,0 days 06:00:05,1,2,8,7,3,3
2,F_2022_09_21.csv,0 days 06:29:14,1,1,5,4,3,2
3,F_2022_09_23.csv,0 days 06:14:49,1,1,2,1,2,1
4,F_2022_08_22.csv,0 days 06:12:46,1,1,2,1,2,1
5,F_2022_08_22.csv,0 days 06:05:03,2,1,2,1,2,1
6,F_2022_08_01.csv,0 days 06:14:17,1,1,8,7,2,2
7,F_2022_08_03.csv,0 days 06:00:32,2,1,2,1,2,1
8,F_2022_09_23.csv,0 days 06:00:10,2,2,13,12,2,2
9,F_2022_09_22.csv,0 days 06:00:30,2,2,6,5,2,2


In [9]:
spaced_aptm_with_state[spaced_aptm_with_state["start_hour"] >= 18][["day", "start_time", "station", "in_station", "started_after", "in_after", "calf_number_started_after", "calf_number_in_after"]].head(20)

,day,start_time,station,in_station,started_after,in_after,calf_number_started_after,calf_number_in_after


In [27]:
# records_sorted["start_hour"].unique()
records_sorted = records_sorted.reset_index(drop=True)

In [28]:
records_sorted[(records_sorted["start_hour"] > 19) & ( ~ records["day"].isin(["F_2022_08_26.csv"]))][["day", "start_time", "station", "in_station", "started_after", "in_after", "calf_number_started_after", "calf_number_in_after"]].head(20)

/tmp/ipykernel_3052045/3297643609.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  records_sorted[(records_sorted["start_hour"] > 19) & ( ~ records["day"].isin(["F_2022_08_26.csv"]))][["day", "start_time", "station", "in_station", "started_after", "in_after", "calf_number_started_after", "calf_number_in_after"]].head(20)


,day,start_time,station,in_station,started_after,in_after,calf_number_started_after,calf_number_in_after
970,F_2022_08_02.csv,0 days 20:01:57,2,1,2,2,2,2
971,F_2022_08_02.csv,0 days 20:01:57,2,2,9,8,3,2
972,F_2022_08_02.csv,0 days 20:04:11,2,1,2,1,2,1
973,F_2022_08_02.csv,0 days 20:04:11,2,2,9,9,3,3
974,F_2022_08_02.csv,0 days 20:03:29,1,1,1,1,1,1
975,F_2022_08_02.csv,0 days 20:03:29,1,2,10,9,3,3
976,F_2022_08_02.csv,0 days 20:06:17,2,1,1,0,1,0
977,F_2022_08_02.csv,0 days 20:06:17,2,2,9,9,2,2
978,F_2022_08_02.csv,0 days 20:08:58,2,1,1,0,1,0
979,F_2022_08_02.csv,0 days 20:08:58,2,2,10,10,2,2


In [30]:
row

day                          F_2022_08_02.csv
station                                     2
in_station                                  1
start_time                    0 days 20:01:57
end_time                      0 days 20:04:10
after_at                      0 days 20:11:57
before_at                     0 days 19:51:57
started_after                               2
calf_number_started_after                   2
in_after                                    2
calf_number_in_after                        2
start_hour                                 20
Name: 970, dtype: object

In [29]:
row_id = 970
row = records_sorted.iloc[row_id]
minutes = 10 * 60  
row_day = row["day"]
station = row["station"]
# station = spaced_aptm_with_state.iloc[row_id]["in_station"]
row_feeder  = open_feeder(os.path.join(dir_path, row_day), columns)
row_feeder[(station == row_feeder["station"]) & (row["start_time"] < row_feeder["start_time"]) & (row["start_time"] + pd.Timedelta(seconds=minutes) > row_feeder["start_time"])]

,date,start visit,end visit,animal index,animal no.,group,station,housing weight,entitlement act.,feed mode,...,visit wo. entit.,break wo. add.,break w. add.,total MP g.,total milk,failure,warnings,start_time,end_time,timedelta
510,2022/08/02,20:04:11,20:04:21,111,4147,A,2,0,0,till,...,41,0,0,14020,0,0,0,0 days 20:04:11,0 days 20:04:21,10.0
512,2022/08/02,20:06:17,20:06:24,77,4151,A,2,0,2500,max.,...,2,0,0,64481,0,0,0,0 days 20:06:17,0 days 20:06:24,7.0
513,2022/08/02,20:08:58,20:09:51,111,4147,A,2,0,0,till,...,42,0,0,14020,0,0,0,0 days 20:08:58,0 days 20:09:51,53.0
514,2022/08/02,20:09:52,20:10:05,111,4147,A,2,0,0,till,...,42,0,0,14020,0,0,0,0 days 20:09:52,0 days 20:10:05,13.0
515,2022/08/02,20:10:36,20:11:09,111,4147,A,2,0,0,till,...,42,0,0,14020,0,0,0,0 days 20:10:36,0 days 20:11:09,33.0
516,2022/08/02,20:11:10,20:11:17,111,4147,A,2,0,0,till,...,42,0,0,14020,0,0,0,0 days 20:11:10,0 days 20:11:17,7.0
517,2022/08/02,20:11:19,20:11:27,111,4147,A,2,0,0,till,...,42,0,0,14020,0,0,0,0 days 20:11:19,0 days 20:11:27,8.0
519,2022/08/02,20:11:32,20:11:49,111,4147,A,2,0,0,till,...,42,0,0,14020,0,0,0,0 days 20:11:32,0 days 20:11:49,17.0
520,2022/08/02,20:11:50,20:13:18,113,4149,A,2,0,363,till,...,4,3,0,98,0,0,0,0 days 20:11:50,0 days 20:13:18,88.0


In [31]:
row_id = 970
row = records_sorted.iloc[row_id]
minutes = 10 * 60  
row_day = row["day"] 
# station = row["station"]
station = row["in_station"]
row_feeder  = open_feeder(os.path.join(dir_path, row_day), columns)
row_feeder[(station == row_feeder["station"]) & (row["start_time"] < row_feeder["start_time"]) & (row["start_time"] + pd.Timedelta(seconds=minutes) > row_feeder["start_time"])]

,date,start visit,end visit,animal index,animal no.,group,station,housing weight,entitlement act.,feed mode,...,visit wo. entit.,break wo. add.,break w. add.,total MP g.,total milk,failure,warnings,start_time,end_time,timedelta
511,2022/08/02,20:03:29,20:05:02,116,4158,A,1,0,2000,from,...,0,5,0,25587,0,0,0,0 days 20:03:29,0 days 20:05:02,93.0
518,2022/08/02,20:05:03,20:11:29,127,4159,A,1,0,2000,from,...,1,8,0,22794,0,0,0,0 days 20:05:03,0 days 20:11:29,386.0


In [66]:
row_id = 2
minutes = 10 * 60  
row_day = spaced_aptm_with_state.iloc[row_id]["day"]
station = spaced_aptm_with_state.iloc[row_id]["station"]
# station = spaced_aptm_with_state.iloc[row_id]["in_station"]
row_feeder  = open_feeder(os.path.join(dir_path, row_day), columns)
row_feeder[(station == row_feeder["station"]) & (spaced_aptm_with_state.iloc[row_id]["start_time"] < row_feeder["start_time"]) & (spaced_aptm_with_state.iloc[row_id]["start_time"] + pd.Timedelta(seconds=minutes) > row_feeder["start_time"])]

,date,start visit,end visit,animal index,animal no.,group,station,housing weight,entitlement act.,feed mode,...,visit wo. entit.,break wo. add.,break w. add.,total MP g.,total milk,failure,warnings,start_time,end_time,timedelta
172,2022/09/21,06:32:48,06:33:14,107,8401,A,1,0,0,BLOCKED,...,0,0,0,2381,0,0,0,0 days 06:32:48,0 days 06:33:14,26.0
173,2022/09/21,06:33:20,06:33:36,107,8401,A,1,0,0,BLOCKED,...,0,0,0,2381,0,0,0,0 days 06:33:20,0 days 06:33:36,16.0
176,2022/09/21,06:34:07,06:35:21,93,8390,A,1,0,0,till,...,4,0,0,50790,0,0,0,0 days 06:34:07,0 days 06:35:21,74.0
177,2022/09/21,06:35:45,06:35:59,93,8390,A,1,0,0,till,...,4,0,0,50790,0,0,0,0 days 06:35:45,0 days 06:35:59,14.0
191,2022/09/21,06:38:02,06:45:23,106,8400,A,1,0,2000,BLOCKED,...,0,0,0,4906,0,0,0,0 days 06:38:02,0 days 06:45:23,441.0


In [75]:
spaced_aptm_with_state.to_csv('/data/data_calves/konrad/feeder_aptms_spaced_metadata.csv', index=False)